In [154]:
import pandas as pd
import numpy as np
import jieba
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.metrics import accuracy_score

In [155]:
train = pd.read_csv("train.csv",delimiter="\t",encoding="utf-8")
test = pd.read_csv("test.csv",delimiter="\t",encoding="utf-8")

In [156]:
train["stance"].unique()

array(['FAVOR', 'AGAINST', 'NONE'], dtype=object)

In [157]:
l2n = {"FAVOR":2,"AGAINST":1,"NONE":0}
n2l = {2:"FAVOR",1:"AGAINST",0:"NONE"}

In [158]:
train["stance"] = train["stance"].map(l2n)

In [159]:
def stopwordslist():
    stopwords = [line.strip() for line in open('cn_stopwords.txt',encoding='UTF-8').readlines()]
    return stopwords

def removepre(sentence):
    pattern = re.compile("[^a-zA-Z0-9\u4E00-\u9FA5]")
    sentence = pattern.sub("",sentence)
    sentence = sentence.replace("#","")
    sentence = sentence.replace("【","")
    sentence = sentence.replace("】","")
    return sentence


def removenumber(sentence):
    pattern = re.compile("[\d+]")
    sentence = pattern.sub("",sentence)
    return sentence
    
        
def seg_depart(sentence):
     # 对文档中的每一行进行中文分词
    sentence_depart = jieba.cut(sentence.strip())
     # 创建一个停用词列表
    stopwords = stopwordslist()
    # 输出结果为outstr
    outstr = ''
   # 去停用词
    for word in sentence_depart:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr  

In [160]:
train["cut"] = train["text"].apply(removepre)
test["cut"] = test["text"].apply(removepre)

In [161]:
train["cut"] = train["cut"].apply(removenumber)
test["cut"] = train["cut"].apply(removenumber)

In [162]:
train["cut"] = train["cut"].apply(seg_depart)
test["cut"] = train["cut"].apply(seg_depart)

In [163]:
train_text = list(train["cut"].values)
test_text = list(test["cut"].values)
summary = train_text+test_text

In [129]:
train["cut"][2]

'春节 放鞭炮 中央气象台 首次 发布 烟花爆竹 燃放 气象 指数 明天 除夕 年 味 越发 浓郁 今早 中央气象台 发布 国家级 烟花爆竹 燃放 气象 指数 预报 预计 月 日 白天 夜间 河北 中南部 河南 南部 湖北 中东部 湖南 西北部 四川盆地 中南部 部分 地区 气象条件 适宜 燃放烟花 爆竹 请 尽量 不要 燃放 '

In [130]:
pattern = re.compile("[\d+]")
pattern.sub("",string)

'# 春节 放鞭炮 # 【 中央气象台 首次 发布 烟花爆竹 燃放 气象 指数 】 明天 除夕 年味 越发 浓郁 今早 中央气象台 发布 国家级 烟花爆竹 燃放 气象 指数 预报 预计 月  日 白天 夜间 河北 中南部 河南 南部 湖北 中东部 湖南 西北部 四川盆地 中南部 部分 地区 气象条件 适宜 燃放烟花 爆竹 请 尽量 不要 燃放 '

In [131]:
a = "明天 除夕 年味 越发 浓郁 今早 中央气象台 "
a.replace("#","")

'明天 除夕 年味 越发 浓郁 今早 中央气象台 '

In [132]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator

In [133]:
tf = TfidfVectorizer(lowercase=False,ngram_range=(1,2),min_df=0)
tf.fit(summary)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=1.0, max_features=None,
                min_df=0, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [134]:
X = tf.transform(train_text)
Y = train["stance"].values
X_test = tf.transform(test_text)

In [135]:
from sklearn.feature_extraction.text import CountVectorizer 
max_features = 3000 #We use the most common word
count_vectorizer = CountVectorizer(max_features = max_features, stop_words = "english")
sparce_matrix = count_vectorizer.fit(summary).transform(train_text).toarray()
print("the most using {} words: {}".format(max_features,count_vectorizer.get_feature_names()))

the most using 3000 words: ['amitdaryanani', 'apple', 'applepay', 'cpu', 'doge', 'dtouch', 'gb', 'httptcnftpse', 'httptcnrhpp', 'httptcnrqgpsed', 'httptcnrqldlag', 'ios', 'ip', 'ipadpro', 'iphone', 'iphones', 'iphonese', 'iphonesplus', 'isis', 'mini', 'nfc', 'periscope', 'phonese', 'plus', 'pm', 'ps', 'se', 'tm', 'uniq', 'vivi', '一下', '一个', '一个劲', '一个多月', '一个月', '一人', '一代', '一代人', '一件', '一份', '一众', '一会儿', '一位', '一個', '一出', '一刀', '一刀切', '一切都是', '一到', '一刻', '一副', '一劳永逸', '一半', '一句', '一台', '一同', '一名', '一周', '一味', '一团', '一场', '一堆', '一大', '一天', '一定', '一家', '一家亲', '一家人', '一岁', '一年', '一张', '一律', '一批', '一把', '一挂', '一是', '一条', '一枚', '一模一样', '一次', '一款', '一步', '一段时间', '一点', '一生', '一直', '一禁', '一种', '一秒', '一篇', '一组', '一群', '一胎', '一致', '一起', '一辆', '一边', '一部', '一部分', '一顿', '七点', '万人', '万辆', '万部', '三个', '三亚', '三人行', '三十', '三十万', '三天', '三年', '三星', '三点', '三百余', '三胎', '三轮', '三轮车', '上下班', '上午', '上学', '上市', '上有老', '上海', '上涨', '上班', '上路', '上面', '下一代', '下令', '下午', '下去', '下有孩', '下来', '下次', '下沙', '下滑', '下班', '

#### SVM

In [136]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_text,Y, test_size = 0.1, random_state = 42)

In [137]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [138]:
text_cal = Pipeline([('tfid', TfidfVectorizer(lowercase=False,ngram_range=(1,2),min_df=0)), ('cal', LinearSVC())])

In [139]:
text_cal.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('tfid',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=0, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('cal',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [140]:
prediction = text_cal.predict(x_test)

In [141]:
np.mean(prediction == y_test)

0.6037735849056604

In [142]:
tf = TfidfVectorizer(lowercase=False,ngram_range=(1,2),min_df=0)

In [143]:
tf.fit_transform(test["cut"])

<600x16107 sparse matrix of type '<class 'numpy.float64'>'
	with 22720 stored elements in Compressed Sparse Row format>

In [144]:
n2l = {2:"FAVOR",1:"AGAINST",0:"NONE"}

In [130]:

import csv
with open('key3.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, dialect='excel')
    for index,res in enumerate(text_cal.predict(test["cut"])):
        fin = ""
        if res == 2:
            fin = "FAVOR"
        if res == 1:
            fin = 'AGAINST'
        if res == 0:
            fin = 'NONE'
        writer.writerow([index,fin])

##### 贝叶斯

In [145]:
x_train,  x_test, y_train, y_test = train_test_split(train_text,Y, test_size = 0.1, random_state = 42)

In [146]:
text_cal_1 = Pipeline([('tfid', TfidfVectorizer(lowercase=False,ngram_range=(1,2),min_df=0)), ('nb', MultinomialNB(alpha=0.2,fit_prior=False))])

In [147]:
text_cal_1.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('tfid',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=0, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('nb',
                 MultinomialNB(alpha=0.2, class_prior=None, fit_prior=False))],
         verbose=False)

In [148]:
prediction = text_cal_1.predict(x_test)

In [149]:
np.mean(prediction == y_test)

0.610062893081761

In [147]:
import csv
with open('key4.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f, dialect='excel')
    for index,res in enumerate(text_cal_1.predict(test["cut"])):
        fin = ""
        if res == 2:
            fin = "FAVOR"
        if res == 1:
            fin = 'AGAINST'
        if res == 0:
            fin = 'NONE'
        writer.writerow([index,fin])

#### XgBoost

In [150]:
x_train

['评论 割爱 公民 担当 春节 脚步 日益 临近 刚刚 送走 一场 雾 霾 时 烟花爆竹 放 放 两种 观点 几乎 打成平手 放 少 放鞭炮 看似 一件 简单 小事 需要 公民 意识 觉醒 需要 一位 公民 担当 这种 环保 责任 担当 融入 日常生活 点滴 中 蓝天 请 割爱 ',
 '深圳 禁摩 限电 每个 开车 横冲直撞 不要命 觉得 违反 交通规则 摩托车 上路 每个 养得 小车 现在 油价 多贵 不能 老百姓 着想 有句 话 说 兴 百姓 苦亡 百姓 苦苦 老百姓 开 小车 到时候 说 污染环境 ',
 '开放 二胎 可能 越来越少 老人 能够 帮忙 带 孩子 退休 延迟 能够 退休 保姆 老人 越来越少 必然 造成 雇 保姆 费用 上涨 加上 国家 延长 产假 鼓励 政策 全职 妈妈 势必 增多 全职 妈妈 会 带来 婚姻 问题 说 当下 婚姻法 没有 保护 条例 孩子 好事 不要 恶性循环 ',
 '不守 交规 摩 司机 出过 事故 一刀切 全都 不准 骑 摩托 简单 粗暴 人民 商量余地 接受 媒体 深圳 禁摩 限电 法律依据 手机 新浪网 httptcnRqLdlag ',
 '觉得 se 牛 逼 全是 傻叉 适合 单手 使用 正常 设计 se 缺点 发布 太晚 显得 苹果 滞后 买 不用 现在 c 升级 硬盘 gb 版本 不太想 换 se ',
 '二胎 指南 再生 一个 不得不 考虑 问题 备孕坊 提示 二胎 政策 放宽 准备 迎接 第二个 孩子 到来 家庭 是否 再要 一个 孩子 常常 取决于 家庭 经济 状况 照顾 孩子 方面 问题 考虑 再要 一个 孩子 时 不要 小看 因素 更多备 孕 知识 请 关注 备孕坊 备孕坊 祝 你好 孕 ',
 '强烈 支持 深圳 交警 限制 措施 出台 符合 大部分 利益 可能 所有人 满意 自私自利 商家 根本 电动车 商家 勾结 段子 手写 颠倒黑白 文章 悲情 牌 说 快递 员 辞职 潮 龙岗 快递 业 根本 没有 影响 ',
 'IphoneSE 今天 拿到 手 系统 运行 不错 手感 质感 方面 不及 系下 不及 S 使用 感受 ',
 '春节 少 放鞭炮 春节 放鞭炮 开着 电脑 只能 听见 鞭炮声 觉得 放鞭炮 不了 年 更何况 这种 居民区 不能自己 挑个 空旷 地方 爱 放 发 ',
 

In [151]:
from xgboost import XGBClassifier
params ={'learning_rate': 0.1,
          'max_depth': 5,
          'num_boost_round':20,
          'objective': 'multi:softmax',
          'random_state': 27,
          'silent':0,
          'num_class':4,
         'num_parallel_tree':3
        }


text_cal_1 = Pipeline([('tfid', TfidfVectorizer(lowercase=False,ngram_range=(1,2),min_df=0)), ('xg', XGBClassifier(max_depth=5,objective="multi:softmax",num_parallel_tree=2,n_jobs=1,subsample=0.8))])

In [152]:
text_cal_1.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('tfid',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=0, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                               interaction_constraints=None,
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=5, min_child_weight=1, missing=nan,
                               monotone_constraints=None, n_estimators=100,
            

In [153]:
prediction = text_cal_1.predict(x_test)
np.mean(prediction == y_test)

0.5974842767295597